In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import os
#import gc
import decoupler as dc
#import regex as re
#import decimal as dec
#from scipy.sparse import csr_matrix
#from scipy.sparse import csc_matrix


In [2]:
adata=ad.read_h5ad('/home/jan/R-projects/reheat2_pilot/data/celltype_data/Fibs/fibs_integrated_sub_reg.h5ad' )


In [3]:
## set up the variables 

#define the column where cluster labels are stored
cluster_col = "leiden_0.5"

#just to save some memory space redo the subsetting to the cell type of interest
cell_type_oi= "Fib"

In [4]:
#data.obs['sample_id'] + '_' + adata.obs['leiden_0.7']
adata.obs['sample_state'] = adata.obs['sample_id'].astype(str) + '_' + adata.obs['leiden_0.5'].astype(str)

In [5]:
adata.obs

,sample_id,cell_type,disease_code,heart_failure,batch,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,leiden_0.4,leiden_0.5,leiden_0.6,leiden_0.7,leiden_0.8,sample_state
TACACCCAGTTCCATG-1-0,P1622,Fib,NF,NF,Chaffin2022,2241,2240,3917.0,0.0,0.000000,0,0,0,1,1,P1622_0
CCACTTGGTGACACGA-1-0,P1622,Fib,NF,NF,Chaffin2022,2246,2246,3815.0,0.0,0.000000,0,0,0,1,4,P1622_0
ACTATGGAGTCACTGT-1-0,P1622,Fib,NF,NF,Chaffin2022,1957,1957,3711.0,0.0,0.000000,2,2,2,2,2,P1622_2
TCGAACACAGAGTAAT-1-0,P1622,Fib,NF,NF,Chaffin2022,2180,2180,3618.0,0.0,0.000000,2,2,2,2,2,P1622_2
ATGACCAAGACTTCAC-1-0,P1622,Fib,NF,NF,Chaffin2022,1984,1984,3564.0,0.0,0.000000,2,2,2,2,2,P1622_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGGAATTTCCAAACCA-1-14,P1801,Fib,NF,NF,Simonson2023,536,535,628.0,3.0,0.477707,2,3,3,3,3,P1801_3
GTCATTTAGCGTCGAA-1-14,P1801,Fib,NF,NF,Simonson2023,537,537,626.0,1.0,0.159744,0,0,0,1,1,P1801_0
TGGGCTGCAGCTTCGG-1-14,P1801,Fib,NF,NF,Simonson2023,527,527,604.0,0.0,0.000000,0,0,0,1,1,P1801_0
GGTTAACGTTAGCGGA-1-14,P1801,Fib,NF,NF,Simonson2023,499,499,577.0,0.0,0.000000,0,0,0,1,1,P1801_0


In [6]:
adata.X = adata.layers["counts"]

In [7]:
#print("getting pseudobulks:", files[i])
padata = dc.get_pseudobulk(adata, sample_col='sample_id', groups_col="leiden_0.5", min_prop=0, min_smpls=0)

pb_dat = pd.DataFrame(padata.X)
pb_dat.columns = padata.var.index.values
pb_dat.index = padata.obs.index.values

/home/jan/programs/mambaforge/envs/scpy/lib/python3.11/site-packages/decoupler/utils_anndata.py:181: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cols = obs.groupby([sample_col, groups_col]).apply(lambda x: x.apply(lambda y: len(y.unique()) == 1)).all(0)
/home/jan/programs/mambaforge/envs/scpy/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [12]:
pb_dat


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
D1_0,3.0,4.0,1.0,34.0,0.0,1.0,38.0,3.0,74.0,8.0,...,11.0,4.0,3.0,0.0,0.0,38.0,3.0,47.0,32.0,48.0
D2_0,4.0,9.0,1.0,87.0,2.0,4.0,42.0,4.0,120.0,23.0,...,22.0,11.0,1.0,3.0,1.0,123.0,4.0,84.0,40.0,138.0
D4_0,19.0,22.0,5.0,178.0,4.0,14.0,120.0,5.0,107.0,51.0,...,25.0,12.0,2.0,3.0,5.0,97.0,3.0,85.0,68.0,104.0
D5_0,8.0,9.0,2.0,147.0,3.0,10.0,94.0,4.0,132.0,45.0,...,24.0,20.0,5.0,3.0,2.0,63.0,5.0,98.0,50.0,108.0
D6_0,10.0,6.0,0.0,78.0,0.0,2.0,36.0,0.0,166.0,22.0,...,9.0,8.0,0.0,3.0,2.0,29.0,0.0,29.0,26.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TWCM-13-285_5,0.0,2.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,6.0
TWCM-13-36_5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,5.0
TWCM-13-96_5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,1.0,0.0,0.0,0.0,1.0,3.0,0.0,7.0,3.0,10.0
TWCM-LVAD2_5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0


In [10]:
pb_dat.to_csv("../../output/fib_sub_analysis/all_state_pb_sub_reg.csv")
adata.obs.to_csv("../../output/fib_sub_analysis/all_state_coldata_sub_reg.csv")